<a href="https://colab.research.google.com/github/ajayostawal/MovieLens-Recommendation-System/blob/main/collaborative_filtering_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Import libraries
import numpy as np
import pandas as pd

# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating', 'timestamp'])

# Reading users file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

In [18]:
movies.head()


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193.0,5.0,978300760.0
1,1,661.0,3.0,978302109.0
2,1,914.0,3.0,978301968.0
3,1,3408.0,4.0,978300275.0
4,1,2355.0,5.0,978824291.0


In [20]:
users.head()

,user_id,gender,zipcode,age_desc,occ_desc
0,1,F,48067,Under 18,K-12 student
1,2,M,70072,56+,self-employed
2,3,M,55117,25-34,scientist
3,4,M,02460,45-49,executive/managerial
4,5,M,55455,25-34,writer


In [21]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 927 | Number of movies = 3371


In [23]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,NaN,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,...,3913.0,3914.0,3915.0,3916.0,3917.0,3918.0,3919.0,3920.0,3921.0,3922.0,3923.0,3924.0,3925.0,3926.0,3927.0,3928.0,3929.0,3930.0,3931.0,3932.0,3933.0,3934.0,3935.0,3936.0,3937.0,3938.0,3939.0,3940.0,3941.0,3942.0,3943.0,3944.0,3945.0,3946.0,3947.0,3948.0,3949.0,3950.0,3951.0,3952.0
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
Ratings.values

array([[0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 4., 1., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 5.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
R = Ratings.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [30]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of MovieLens1M dataset is 95.39999999999999%


In [31]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [32]:
sigma = np.diag(sigma)


In [33]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)


In [34]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movie_id,NaN,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,...,3913.0,3914.0,3915.0,3916.0,3917.0,3918.0,3919.0,3920.0,3921.0,3922.0,3923.0,3924.0,3925.0,3926.0,3927.0,3928.0,3929.0,3930.0,3931.0,3932.0,3933.0,3934.0,3935.0,3936.0,3937.0,3938.0,3939.0,3940.0,3941.0,3942.0,3943.0,3944.0,3945.0,3946.0,3947.0,3948.0,3949.0,3950.0,3951.0,3952.0
0,0.011689,3.946247,0.460839,-0.251062,-0.047600,-0.062444,-0.002209,-0.127891,0.257595,0.009140,-0.036057,-0.059798,-0.054669,0.337526,0.073829,-0.099165,0.078012,0.340384,-0.026402,0.061455,0.054094,-0.500608,-0.037631,0.238915,0.506544,0.164746,0.112860,0.027103,-0.091416,-0.027154,-0.038450,0.167202,0.405312,-0.009067,1.985161,0.040755,0.963478,0.030939,0.014025,0.341609,...,0.063147,0.097956,-0.062222,0.154126,-0.292479,-0.193394,-0.095691,-0.145593,-0.048135,-0.066188,-0.045541,-0.034296,-0.076718,-0.222252,-0.178038,-0.122147,-0.026363,-0.210910,-0.037194,-0.278611,-0.048795,-0.076705,-0.022956,-0.129694,0.000269,0.016230,-0.010208,0.006861,0.061678,-0.000861,-0.046872,0.022492,0.089842,-0.057584,-0.262105,0.969808,-0.124268,0.023270,-0.031548,0.132781
1,0.011878,0.750159,0.190558,0.550928,-0.019526,-0.124847,0.594273,0.201183,-0.056792,0.283563,1.130345,1.236212,-0.170739,-0.078881,-0.031843,0.045351,0.271952,0.998732,0.089299,0.095486,-0.025624,2.104907,0.145748,-0.050287,-0.179807,0.713215,0.049180,0.062244,-0.124153,-0.241262,-0.148557,-0.198208,0.714306,-0.028727,1.098007,0.069684,1.330191,-0.001285,-0.008952,0.016003,...,-0.093128,-0.107199,-0.238824,0.845129,-0.017650,-0.025124,0.012706,-0.215991,-0.019149,-0.033464,0.069533,-0.018725,-0.133417,0.234667,0.330314,0.162887,0.217742,0.241638,-0.011017,0.268800,0.076542,0.063568,0.092380,0.125086,-0.133122,0.011038,0.019904,0.014411,-0.013122,-0.055778,-0.164973,-0.001730,0.001902,-0.048811,-0.149059,0.333168,-0.153091,-0.202350,-0.206140,-0.430157
2,0.003545,1.694431,0.376261,0.009194,0.053631,0.079187,0.375279,-0.372423,0.019754,0.001599,0.391944,0.199027,0.002610,0.001049,0.079664,-0.075318,-0.239378,-0.026104,-0.016475,0.194962,0.097290,0.723569,0.015062,-0.034634,0.329591,0.211998,0.121559,-0.014470,0.055004,-0.287078,-0.117620,0.092688,-0.365737,-0.017890,0.675624,-0.056683,0.204446,0.006078,0.005433,0.498213,...,0.032877,-0.036171,-0.014510,-0.108793,0.064050,0.091578,0.135991,-0.037498,0.049999,0.013824,-0.045684,-0.002190,0.162803,-0.073761,-0.075311,0.326442,0.057643,0.012019,-0.009886,-0.018952,0.002771,-0.028839,0.016662,0.074032,-0.092722,0.085658,0.048582,0.035363,0.044102,0.023405,-0.045345,0.003932,0.056732,0.026867,-0.019928,0.285571,-0.035369,0.019142,0.041919,-0.180876
3,0.001160,0.131313,-0.099086,0.093032,0.092405,0.115322,0.195315,-0.069621,-0.035315,-0.013385,0.251434,-0.145280,-0.006003,0.042754,0.078545,-0.017381,-0.230504,-0.122143,0.040198,0.087564,-0.006959,0.338480,0.024845,-0.029446,-0.035806,0.116497,0.069102,0.002815,-0.038799,-0.000114,-0.034668,0.036764,0.048733,-0.015151,-0.138915,0.046583,0.016662,0.008899,0.002204,-0.006259,...,-0.016145,-0.035950,-0.047396,0.032131,-0.051742,-0.067367,0.015278,-0.018634,0.081917,0.059424,-0.017745,0.045400,0.041049,-0.047623,-0.008145,0.069298,-0.048944,0.090148,-0.013415,0.121142,-0.021270,-0.052170,0.013066,-0.025944,-0.124182,0.022642,0.027235,0.024574,0.047512,0.025449,0.000853,0.001334,0.000605,-0.066625,-0.048845,-0.083582,0.048479,-0.011112,0.002262,-0.131856
4,0.019215,0.679623,0.124534,0.212692,0.420999,-0.022758,1.019047,-0.630520,0.157039,-0.159560,0.198473,-0.173108,-0.039938,-0.067970,0.298403,0.029095,1.262899,0.798973,0.208889,0.037674,-0.155083,1.224993,-0.056319,-0.134951,0.194145,1.782867,-0.015576,-0.225015,0.243090,0.934841,0.155611,-0.127082,2.128656,0.035499,2.493914,0.276399,1.461688,0.046586,-0.006266,1.044913,...,-0.056189,0.252479,0.191517,-0.618141,0.485838,0.031112,0.035577,-0.043548,0.115868,0.

In [36]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [43]:
already_rated, predictions = recommend_movies(preds, 310, movies, ratings,5)


User 310 has already rated 223 movies.
Recommending highest 5 predicted ratings movies not already rated.


In [44]:

# Top 20 movies that User 1310 has rated 
already_rated.head(20)

,user_id,movie_id,rating,timestamp,title,genres
0,310,719.0,5.0,976478304.0,Multiplicity (1996),Comedy
43,310,2236.0,5.0,976477531.0,Simon Birch (1998),Drama
133,310,3461.0,5.0,976481230.0,Lord of the Flies (1963),Adventure|Drama|Thriller
124,310,1777.0,5.0,976477459.0,"Wedding Singer, The (1998)",Comedy|Romance
122,310,2572.0,5.0,976477596.0,10 Things I Hate About You (1999),Comedy|Romance
114,310,1835.0,5.0,976478485.0,City of Angels (1998),Romance
1,310,575.0,5.0,976483399.0,"Little Rascals, The (1994)",Children's|Comedy
109,310,2485.0,5.0,976481736.0,She's All That (1999),Comedy|Romance
107,310,2629.0,5.0,976478168.0,"Love Letter, The (1999)",Comedy|Romance
102,310,1821.0,5.0,976481525.0,"Object of My Affection, The (1998)",Comedy|Romance


In [45]:
predictions


,movie_id,title,genres
1112,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
294,318,"Shawshank Redemption, The (1994)",Drama
96,110,Braveheart (1995),Action|Drama|War
313,339,While You Were Sleeping (1995),Comedy|Romance
3604,3897,Almost Famous (2000),Comedy|Drama


In [49]:
pip install scikit-surprise

     |████████████████████████████████| 11.8MB 246kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617644 sha256=e9d0adcfbb043f5a6f92554a3f64e063236c7dea0ef7494f5569c04dadc0b889
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [59]:
# Import libraries from Surprise package
import surprise
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

reader = Reader()

data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)


# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.7890  nan     1.7946  1.8024  1.7958  nan     nan     
MAE (testset)     1.4035  nan     1.4147  1.4174  1.4133  nan     nan     
Fit time          7.78    6.96    6.71    6.92    6.90    7.05    0.38    
Test time         0.33    0.25    0.34    0.39    0.23    0.31    0.06    


{'fit_time': (7.784466028213501,
  6.958837509155273,
  6.706732749938965,
  6.916215658187866,
  6.899669885635376),
 'test_mae': array([1.40346785,        nan, 1.41467915, 1.41744429, 1.41330084]),
 'test_rmse': array([1.7890276 ,        nan, 1.79458505, 1.80239897, 1.79580911]),
 'test_time': (0.33304524421691895,
  0.2528088092803955,
  0.3355116844177246,
  0.38561201095581055,
  0.22887015342712402)}

In [77]:
ratings[ratings['user_id'] == 310]


,user_id,movie_id,rating,timestamp
46338,310,719.0,5.0,9.764783e+08
46339,310,575.0,5.0,9.764834e+08
46340,310,3791.0,5.0,9.764782e+08
46341,310,2125.0,3.0,9.764814e+08
46342,310,2052.0,4.0,9.764833e+08
...,...,...,...,...
46556,310,2978.0,4.0,1.012082e+09
46557,310,708.0,4.0,9.764815e+08
46558,310,562.0,5.0,9.764826e+08
46559,310,497.0,4.0,9.764814e+08


In [78]:
algo.predict(310, 1994)


Prediction(uid=310, iid=1994, r_ui=None, est=5, details={'was_impossible': False})